<a href="https://colab.research.google.com/github/MarcosMota/AnaliseDeSentimento/blob/master/Reviews_de_Ecommerce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Conjunto de Dados

#Rotulação

#Pre Processamento

#Treinamento

#Validação

#Conclusão